In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
import json
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification


In [17]:
#open the data file
with open ('./data/data.json', 'r', encoding="utf8") as f:
    datastore = json.load(f)
requests = []
labels = []
full = []

In [18]:
for item in datastore:
    requests.append(item['request'])
    parameters = [item['is_order'],item['is_refund'],item['is_status'],item['is_date_change'],item['is_dest_change'],item['is_weather'],item['is_allowed']]
    full.append(parameters)
    labels.append(parameters.index(1))
print(labels)


[3, 5, 4, 3, 2, 4, 5, 4, 6, 6, 3, 2, 1, 0, 6, 2, 2, 4, 3, 5, 4, 2, 4, 6, 6, 3, 2, 1, 0, 6, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6]


In [19]:
#import hebert to tokenize the data
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

In [20]:
#tokenize the data
max_length = 128
encoded_data = tokenizer(requests, truncation=True, padding=True, max_length=max_length, return_tensors='tf')

# Convert BatchEncoding to dictionary
encoded_data = {key: encoded_data[key].numpy() for key in encoded_data}

In [21]:
# Convert labels to one-hot encoding
num_classes = 7
labels_one_hot = tf.keras.utils.to_categorical(labels, num_classes=num_classes)
print(labels_one_hot)

[[0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.

In [22]:
#split the data into training and testing
train_inputs, test_inputs, train_labels, test_labels = train_test_split(encoded_data['input_ids'], labels_one_hot, test_size=0.2, random_state=42)


In [23]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=num_classes)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [25]:
# Train the model
batch_size = 32
epochs = 3
model.fit(train_inputs, train_labels, validation_data=(test_inputs, test_labels), batch_size=batch_size, epochs=epochs)

Epoch 1/3
2/2 [==============================] - 31s 6s/step - loss: 1.9766 - accuracy: 0.1296 - val_loss: 1.8880 - val_accuracy: 0.1429
Epoch 2/3
2/2 [==============================] - 5s 3s/step - loss: 1.9063 - accuracy: 0.2222 - val_loss: 1.8605 - val_accuracy: 0.1429
Epoch 3/3
2/2 [==============================] - 6s 3s/step - loss: 1.8410 - accuracy: 0.3704 - val_loss: 1.8400 - val_accuracy: 0.1429


In [26]:
# Make predictions
predictions = model.predict(encoded_data)
predicted_classes = tf.argmax(predictions.logits, axis=1).numpy()

# Convert predicted classes to original labels
# (assuming your original labels were 1-indexed)
predicted_labels = predicted_classes +1

# Print predictions
# print(f"original Label: {labels}")
# for sentence, predicted_label in zip(requests, predicted_labels):
#     print(f"Sentence: {sentence}")
#     print(f"Predicted Label: {predicted_label}")

#print model accuracy
accuracy = np.mean(labels == predicted_labels)
print(f"Accuracy: {accuracy:.2%}")
    

3/3 [==============================] - 4s 290ms/step
Accuracy: 13.24%
